# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def load_data(nhl = False, cit = False):
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    if nhl:
        return nhl_df
    elif cit:
        return cities

def nhl_correlation(): 
    
    nhl_df = load_data(nhl=True)
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    nhl_df = nhl_df[(nhl_df["team"] != "Atlantic Division") & (nhl_df["team"] != "Metropolitan Division") & (nhl_df["team"] != "Central Division") & (nhl_df["team"] != "Pacific Division")][["team","W","L"]]
    nhl_df["W/L"] = nhl_df["W"].astype(float) / (nhl_df["W"].astype(float) + nhl_df["L"].astype(float))
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.strip('*'))
    nhl_df.index = range(0,len(nhl_df))
    
    cities = load_data(cit=True)
    cities = load_data(cit=True)
    cities.drop(labels = ["NFL", "MLB", "NBA"], axis = 1, inplace = True)
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split("[")[0])
    cities = cities[(cities["NHL"] != "—") & (cities["NHL"] != "")]
    ranger = cities.iloc[0]

    data = {"Metropolitan area":["New York City", "New York City","New York City"],
              "Population (2016 est.)[8]":["20153634", "20153634", "20153634"],
              "NHL":["Rangers", "Islanders", "Devils"]}
    data1 = {"Metropolitan area":["Los Angeles", "Los Angeles"],
              "Population (2016 est.)[8]":["13310447", "13310447"],
              "NHL":["Kings", "Ducks"]}

    data = pd.DataFrame.from_dict(data, orient='index')
    data1 = pd.DataFrame.from_dict(data1, orient='index')
    cities = pd.concat([cities, data.T, data1.T])
    cities.index = range(0,len(cities.index))
    cities.drop([0,1],inplace=True)
    cities.columns = ["Metropolitan area",'Population','NHL']
    cities['Population'] = cities['Population'].astype(int)
    cities.index = range(0,len(cities.index))
    
    df = pd.concat([cities, nhl_df[['team','W/L']]],axis=1)
    df[['copy team','copy W/L']] = df[["team","W/L"]]

    for idx, i in enumerate(df['NHL']):
        for jdx, j in enumerate(df['copy team']):
            if df['NHL'].iloc[idx] in df['copy team'].iloc[jdx]:
                df['team'].iloc[idx] = df['copy team'].iloc[jdx]
                df['W/L'].iloc[idx] = df['copy W/L'].iloc[jdx]

    df.drop(['copy team','copy W/L','NHL'], axis = 1, inplace = True)
    df.set_index('Metropolitan area', inplace=True)
    
    df_g = df.groupby("Metropolitan area").agg({"Population":np.mean,"W/L":np.mean})
    
    population_by_region = list(df_g['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(df_g['W/L']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [8]:
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def load_data(nba = False, cit = False):
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    if nba:
        return nba_df
    elif cit:
        return cities

def nba_correlation():
    nba_df = load_data(nba=True)
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df["W/L"] = nba_df["W"].astype(float) / (nba_df["W"].astype(float) + nba_df["L"].astype(float))
    nba_df = nba_df[["team","W/L"]]
    nba_df['team'] = nba_df['team'].apply(lambda x: x.split('*')[0])
    nba_df['team'] = nba_df['team'].apply(lambda x: x.split('(')[0])
    
    cities = load_data(cit=True)
    cities.drop(["NFL", "MLB", "NHL"], axis=1, inplace=True)
    cities.columns = ["Metropolitan area","Population","NBA"]
    cities["NBA"] = cities["NBA"].apply(lambda x: x.split("[")[0])
    cities = cities[(cities["NBA"] != "—") & (cities["NBA"] != "")]


    dic = {"Metropolitan area":["New York City", "New York City"],
          "Population":["20153634", "20153634"],
          "NBA":["Knicks","Nets"]}
    dic1 = {"Metropolitan area":["Los Angeles", "Los Angeles"],
          "Population":["13310447", "13310447"],
          "NBA":["Lakers","Clippers"]}

    dic = pd.DataFrame.from_dict(dic)
    dic1 = pd.DataFrame.from_dict(dic1)

    cities = pd.concat([cities, dic, dic1])
    cities.index = range(0,len(cities.index))
    cities.drop([0,1], inplace = True)
    cities.index = range(0,len(cities.index))
    
    cities = pd.concat([cities, nba_df], axis =1)
    cities[['copy team','copy W/L']] = cities[["team","W/L"]]
    
    for idx, i in enumerate(cities['NBA']):
        for jdx, j in enumerate(cities['copy team']):
            if cities['NBA'].iloc[idx] in cities['copy team'].iloc[jdx]:
                cities['team'].iloc[idx] = cities['copy team'].iloc[jdx]
                cities['W/L'].iloc[idx] = cities['copy W/L'].iloc[jdx]

    cities.drop(['copy team','copy W/L','NBA'], axis = 1, inplace = True)
    cities.set_index('Metropolitan area', inplace=True)
    cities["Population"] = cities["Population"].astype(int)
    
    df = cities.groupby("Metropolitan area").agg({"Population":np.mean,
                                             "W/L":np.mean})
    
    population_by_region = list(df["Population"]) # pass in metropolitan area population from cities
    win_loss_by_region = list(df["W/L"]) # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [11]:
nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [12]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def load_data(mlb = False, cit = False):
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    if mlb:
        return mlb_df
    elif cit:
        return cities

def mlb_correlation(): 
    mlb_df = load_data(mlb=True)
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    mlb_df["W/L"] = mlb_df["W"].astype(float) / (mlb_df["W"].astype(float) + mlb_df["L"].astype(float))
    mlb_df.drop(["W-L%","GB","year","League","W","L"], axis=1, inplace=True)
    
    cities = load_data(cit=True)
    cities.drop(["NFL","NBA","NHL"], axis=1, inplace = True)
    cities.columns = ["Metropolitan area","Population","MLB"]
    cities["MLB"] = cities["MLB"].apply(lambda x: x.split('[')[0])
    cities = cities[(cities["MLB"] != "—") & (cities["MLB"] != "")]
    cities.index = range(0,len(cities.index))
    
    dic = {"Metropolitan area":["New York City","New York City","Los Angeles","Los Angeles","San Francisco Bay Area","San Francisco Bay Area","Chicago","Chicago"],
      "Population":["20153634","20153634","13310447","13310447","6657982","6657982","9512999","9512999"],
      "MLB":["Yankees","Mets","Dodgers","Angels","Giants","Athletics","Cubs","White Sox"]}

    dic = pd.DataFrame.from_dict(dic)

    df = pd.concat([cities,dic])
    df.index = range(0,len(df.index))
    df.drop([0,1,2,3], inplace=True)
    df.index = range(0,len(df.index))
    df = pd.concat([df,mlb_df],axis=1)
    df[["copy team","copy W/L"]] = df[["team","W/L"]]
    
    for idx, i in enumerate(df['MLB']):
        for jdx, j in enumerate(df['copy team']):
            if df['MLB'].iloc[idx] in df['copy team'].iloc[jdx]:
                df['team'].iloc[idx] = df['copy team'].iloc[jdx]
                df['W/L'].iloc[idx] = df['copy W/L'].iloc[jdx]

    df.drop(['copy team','copy W/L','MLB'], axis = 1, inplace = True)
    df.set_index('Metropolitan area', inplace=True)
    df["Population"] = df['Population'].astype(int)
    
    df_g = df.groupby("Metropolitan area").agg({"Population":np.mean,"W/L":np.mean})
    
    population_by_region = list(df_g["Population"]) # pass in metropolitan area population from cities
    win_loss_by_region = list(df_g["W/L"]) # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [13]:
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def load_data(nfl = False, cit = False):
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    if nfl:
        return nfl_df
    elif cit:
        return cities
    
def nfl_correlation(): 
    nfl_df = load_data(nfl=True)
    nfl_df = nfl_df[(nfl_df["year"] == 2018) & (nfl_df["team"] != "AFC East") & (nfl_df["team"] != "AFC North") & (nfl_df["team"] != "AFC South") & (nfl_df["team"] != "AFC West") & (nfl_df["team"] != "NFC East") & (nfl_df["team"] != "NFC North") & (nfl_df["team"] != "NFC South") & (nfl_df["team"] != "NFC West")]
    nfl_df["W/L"] = nfl_df["W"].astype(int) / (nfl_df["W"].astype(int) + nfl_df["L"].astype(int))
    nfl_df = nfl_df[["team","W/L"]]
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split("*")[0])
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split("+")[0])
    nfl_df.index = range(0,len(nfl_df.index))
    
    cities = load_data(cit=True)
    cities.drop(["MLB","NBA","NHL"], axis=1, inplace = True)
    cities.columns = ["Metropolitan area","Population","NFL"]
    cities["NFL"] = cities["NFL"].apply(lambda x: x.split('[')[0])
    cities = cities[(cities["NFL"] != "—") & (cities["NFL"] != "") & (cities["NFL"] != "— ")]
    cities.index = range(0,len(cities.index))
    
    dic = {"Metropolitan area":["New York City","New York City","Los Angeles","Los Angeles","San Francisco Bay Area","San Francisco Bay Area"],
      "Population":["20153634","20153634","13310447","13310447","6657982","6657982"],
      "NFL":["Giants","Jets","Rams","Chargers","49ers","Raiders"]}

    dic = pd.DataFrame.from_dict(dic)

    df = pd.concat([cities,dic])
    df.index = range(0,len(df.index))
    df.drop([0,1,2], inplace=True)
    df.index = range(0,len(df.index))
    
    df = pd.concat([df,nfl_df],axis=1)
    df[["copy team","copy W/L"]] = df[["team","W/L"]]
    
    for idx, i in enumerate(df['NFL']):
        for jdx, j in enumerate(df['copy team']):
            if df['NFL'].iloc[idx] in df['copy team'].iloc[jdx]:
                df['team'].iloc[idx] = df['copy team'].iloc[jdx]
                df['W/L'].iloc[idx] = df['copy W/L'].iloc[jdx]

    df.drop(['copy team','copy W/L','NFL'], axis = 1, inplace = True)
    df.set_index('Metropolitan area', inplace=True)
    df["Population"] = df['Population'].astype(int)
    
    df_g = df.groupby("Metropolitan area").agg({"Population":np.mean,"W/L":np.mean})
    
    population_by_region = list(df_g["Population"]) # pass in metropolitan area population from cities
    win_loss_by_region = list(df_g["W/L"]) # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [16]:
nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [19]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def load_data(mlb= False, nhl = False, nba = False, nfl = False, cit = False):
    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    if mlb:
        return mlb_df
    elif nhl:
        return nhl_df
    elif nba:
        return nba_df
    elif nfl:
        return nfl_df
    elif cit:
        return cities
    
def nhl_correlation(): 
    
    nhl_df = load_data(nhl=True)
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    nhl_df = nhl_df[(nhl_df["team"] != "Atlantic Division") & (nhl_df["team"] != "Metropolitan Division") & (nhl_df["team"] != "Central Division") & (nhl_df["team"] != "Pacific Division")][["team","W","L"]]
    nhl_df["W/L"] = nhl_df["W"].astype(float) / (nhl_df["W"].astype(float) + nhl_df["L"].astype(float))
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.strip('*'))
    nhl_df.index = range(0,len(nhl_df))
    
    cities = load_data(cit=True)
    cities.drop(labels = ["NFL", "MLB", "NBA"], axis = 1, inplace = True)
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split("[")[0])
    cities = cities[(cities["NHL"] != "—") & (cities["NHL"] != "")]
    ranger = cities.iloc[0]

    data = {"Metropolitan area":["New York City", "New York City","New York City"],
              "Population (2016 est.)[8]":["20153634", "20153634", "20153634"],
              "NHL":["Rangers", "Islanders", "Devils"]}
    data1 = {"Metropolitan area":["Los Angeles", "Los Angeles"],
              "Population (2016 est.)[8]":["13310447", "13310447"],
              "NHL":["Kings", "Ducks"]}

    data = pd.DataFrame.from_dict(data, orient='index')
    data1 = pd.DataFrame.from_dict(data1, orient='index')
    cities = pd.concat([cities, data.T, data1.T])
    cities.index = range(0,len(cities.index))
    cities.drop([0,1],inplace=True)
    cities.columns = ["Metropolitan area",'Population','NHL']
    cities['Population'] = cities['Population'].astype(int)
    cities.index = range(0,len(cities.index))
    
    df = pd.concat([cities, nhl_df[['team','W/L']]],axis=1)
    df[['copy team','copy W/L']] = df[["team","W/L"]]

    for idx, i in enumerate(df['NHL']):
        for jdx, j in enumerate(df['copy team']):
            if df['NHL'].iloc[idx] in df['copy team'].iloc[jdx]:
                df['team'].iloc[idx] = df['copy team'].iloc[jdx]
                df['W/L'].iloc[idx] = df['copy W/L'].iloc[jdx]

    df.drop(['copy team','copy W/L','NHL'], axis = 1, inplace = True)
    df.set_index('Metropolitan area', inplace=True)
    
    df_nhl = df
    #df.groupby("Metropolitan area").agg({"Population":np.mean,"W/L":np.mean})
    
    return df_nhl

def nba_correlation():
    nba_df = load_data(nba=True)
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df["W/L"] = nba_df["W"].astype(float) / (nba_df["W"].astype(float) + nba_df["L"].astype(float))
    nba_df = nba_df[["team","W/L"]]
    nba_df['team'] = nba_df['team'].apply(lambda x: x.split('*')[0])
    nba_df['team'] = nba_df['team'].apply(lambda x: x.split('(')[0])
    
    cities = load_data(cit=True)
    cities.drop(["NFL", "MLB", "NHL"], axis=1, inplace=True)
    cities.columns = ["Metropolitan area","Population","NBA"]
    cities["NBA"] = cities["NBA"].apply(lambda x: x.split("[")[0])
    cities = cities[(cities["NBA"] != "—") & (cities["NBA"] != "")]


    dic = {"Metropolitan area":["New York City", "New York City"],
          "Population":["20153634", "20153634"],
          "NBA":["Knicks","Nets"]}
    dic1 = {"Metropolitan area":["Los Angeles", "Los Angeles"],
          "Population":["13310447", "13310447"],
          "NBA":["Lakers","Clippers"]}

    dic = pd.DataFrame.from_dict(dic)
    dic1 = pd.DataFrame.from_dict(dic1)

    cities = pd.concat([cities, dic, dic1])
    cities.index = range(0,len(cities.index))
    cities.drop([0,1], inplace = True)
    cities.index = range(0,len(cities.index))
    
    cities = pd.concat([cities, nba_df], axis =1)
    cities[['copy team','copy W/L']] = cities[["team","W/L"]]
    
    for idx, i in enumerate(cities['NBA']):
        for jdx, j in enumerate(cities['copy team']):
            if cities['NBA'].iloc[idx] in cities['copy team'].iloc[jdx]:
                cities['team'].iloc[idx] = cities['copy team'].iloc[jdx]
                cities['W/L'].iloc[idx] = cities['copy W/L'].iloc[jdx]

    cities.drop(['copy team','copy W/L','NBA'], axis = 1, inplace = True)
    cities.set_index('Metropolitan area', inplace=True)
    cities["Population"] = cities["Population"].astype(int)
    
    df_nba = cities
    #cities.groupby("Metropolitan area").agg({"Population":np.mean,"W/L":np.mean})

    return df_nba

def mlb_correlation(): 
    mlb_df = load_data(mlb=True)
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    mlb_df["W/L"] = mlb_df["W"].astype(float) / (mlb_df["W"].astype(float) + mlb_df["L"].astype(float))
    mlb_df.drop(["W-L%","GB","year","League","W","L"], axis=1, inplace=True)
    
    cities = load_data(cit=True)
    cities.drop(["NFL","NBA","NHL"], axis=1, inplace = True)
    cities.columns = ["Metropolitan area","Population","MLB"]
    cities["MLB"] = cities["MLB"].apply(lambda x: x.split('[')[0])
    cities = cities[(cities["MLB"] != "—") & (cities["MLB"] != "")]
    cities.index = range(0,len(cities.index))
    
    dic = {"Metropolitan area":["New York City","New York City","Los Angeles","Los Angeles","San Francisco Bay Area","San Francisco Bay Area","Chicago","Chicago"],
      "Population":["20153634","20153634","13310447","13310447","6657982","6657982","9512999","9512999"],
      "MLB":["Yankees","Mets","Dodgers","Angels","Giants","Athletics","Cubs","White Sox"]}

    dic = pd.DataFrame.from_dict(dic)

    df = pd.concat([cities,dic])
    df.index = range(0,len(df.index))
    df.drop([0,1,2,3], inplace=True)
    df.index = range(0,len(df.index))
    df = pd.concat([df,mlb_df],axis=1)
    df[["copy team","copy W/L"]] = df[["team","W/L"]]
    
    for idx, i in enumerate(df['MLB']):
        for jdx, j in enumerate(df['copy team']):
            if df['MLB'].iloc[idx] in df['copy team'].iloc[jdx]:
                df['team'].iloc[idx] = df['copy team'].iloc[jdx]
                df['W/L'].iloc[idx] = df['copy W/L'].iloc[jdx]

    df.drop(['copy team','copy W/L','MLB'], axis = 1, inplace = True)
    df.set_index('Metropolitan area', inplace=True)
    df["Population"] = df['Population'].astype(int)
    
    df_mlb = df
    #df.groupby("Metropolitan area").agg({"Population":np.mean,"W/L":np.mean})
    
    return df_mlb

def nfl_correlation(): 
    nfl_df = load_data(nfl=True)
    nfl_df = nfl_df[(nfl_df["year"] == 2018) & (nfl_df["team"] != "AFC East") & (nfl_df["team"] != "AFC North") & (nfl_df["team"] != "AFC South") & (nfl_df["team"] != "AFC West") & (nfl_df["team"] != "NFC East") & (nfl_df["team"] != "NFC North") & (nfl_df["team"] != "NFC South") & (nfl_df["team"] != "NFC West")]
    nfl_df["W/L"] = nfl_df["W"].astype(int) / (nfl_df["W"].astype(int) + nfl_df["L"].astype(int))
    nfl_df = nfl_df[["team","W/L"]]
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split("*")[0])
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split("+")[0])
    nfl_df.index = range(0,len(nfl_df.index))
    
    cities = load_data(cit=True)
    cities.drop(["MLB","NBA","NHL"], axis=1, inplace = True)
    cities.columns = ["Metropolitan area","Population","NFL"]
    cities["NFL"] = cities["NFL"].apply(lambda x: x.split('[')[0])
    cities = cities[(cities["NFL"] != "—") & (cities["NFL"] != "") & (cities["NFL"] != "— ")]
    cities.index = range(0,len(cities.index))
    
    dic = {"Metropolitan area":["New York City","New York City","Los Angeles","Los Angeles","San Francisco Bay Area","San Francisco Bay Area"],
      "Population":["20153634","20153634","13310447","13310447","6657982","6657982"],
      "NFL":["Giants","Jets","Rams","Chargers","49ers","Raiders"]}

    dic = pd.DataFrame.from_dict(dic)

    df = pd.concat([cities,dic])
    df.index = range(0,len(df.index))
    df.drop([0,1,2], inplace=True)
    df.index = range(0,len(df.index))
    
    df = pd.concat([df,nfl_df],axis=1)
    df[["copy team","copy W/L"]] = df[["team","W/L"]]
    
    for idx, i in enumerate(df['NFL']):
        for jdx, j in enumerate(df['copy team']):
            if df['NFL'].iloc[idx] in df['copy team'].iloc[jdx]:
                df['team'].iloc[idx] = df['copy team'].iloc[jdx]
                df['W/L'].iloc[idx] = df['copy W/L'].iloc[jdx]

    df.drop(['copy team','copy W/L','NFL'], axis = 1, inplace = True)
    df.set_index('Metropolitan area', inplace=True)
    df["Population"] = df['Population'].astype(int)
    
    df_nfl = df
    #df.groupby("Metropolitan area").agg({"Population":np.mean,"W/L":np.mean})

    return df_nfl

def sports_team_performance():
    
    df_nhl = nhl_correlation()
    df_nba = nba_correlation()
    df_mlb = mlb_correlation()
    df_nfl = nfl_correlation()
    
    df_nhl.drop("Population", axis=1, inplace=True)
    df_nhl.columns = ["NHL","NHL W/L"]

    df_nba.drop("Population", axis=1, inplace=True)
    df_nba.columns = ["NBA","NBA W/L"]

    df_mlb.drop("Population", axis=1, inplace=True)
    df_mlb.columns = ["MLB","MLB W/L"]

    df_nfl.drop("Population", axis=1, inplace=True)
    df_nfl.columns = ["NFL","NFL W/L"]

    df = pd.merge(df_nhl,df_nba, how="outer", left_index=True, right_index=True)
    df = pd.merge(df, df_mlb, how="outer", left_index=True, right_index=True)
    df = pd.merge(df, df_nfl, how="outer", left_index=True, right_index=True)

    df_1= df[["NHL","NHL W/L","NBA","NBA W/L"]]
    df_1.dropna(axis=0,inplace=True)
    df_1 = df_1.drop_duplicates()
    df_1 = df_1.groupby("Metropolitan area").agg({"NHL W/L": np.mean,"NBA W/L": np.mean})

    df_2 = df[["NHL","NHL W/L","MLB","MLB W/L"]]
    df_2.dropna(axis=0,inplace=True)
    df_2 = df_2.drop_duplicates()
    df_2 = df_2.groupby("Metropolitan area").agg({"NHL W/L": np.mean,"MLB W/L": np.mean})

    df_3 = df[["NHL","NHL W/L","NFL","NFL W/L"]]
    df_3.dropna(axis=0,inplace=True)
    df_3 = df_3.drop_duplicates()
    df_3 = df_3.groupby("Metropolitan area").agg({"NHL W/L": np.mean,"NFL W/L": np.mean})

    df_4 = df[["NBA","NBA W/L","MLB","MLB W/L"]]
    df_4.dropna(axis=0,inplace=True)
    df_4 = df_4.drop_duplicates()
    df_4 = df_4.groupby("Metropolitan area").agg({"NBA W/L": np.mean,"MLB W/L": np.mean})

    df_5 = df[["NBA","NBA W/L","NFL","NFL W/L"]]
    df_5.dropna(axis=0,inplace=True)
    df_5 = df_5.drop_duplicates()
    df_5 = df_5.groupby("Metropolitan area").agg({"NBA W/L": np.mean,"NFL W/L": np.mean})

    df_6 = df[["MLB","MLB W/L","NFL","NFL W/L"]]
    df_6.dropna(axis=0,inplace=True)
    df_6 = df_6.drop_duplicates()
    df_6 = df_6.groupby("Metropolitan area").agg({"MLB W/L": np.mean,"NFL W/L": np.mean})
    
    df_list = [df_1, df_2, df_3, df_4, df_5, df_6]
    
    p_values = []

    comparisons = []

    for i in df_list:
        x = i.columns[0]
        y = i.columns[1]
        comparisons.append(stats.ttest_rel(i[x],i[y])[1])
    
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    p_values["NFL"].loc["NBA"], p_values["NBA"].loc["NFL"] = comparisons[4], comparisons[4]
    p_values["NFL"].loc["NHL"], p_values["NHL"].loc["NFL"] = comparisons[2], comparisons[2]
    p_values["NFL"].loc["MLB"], p_values["MLB"].loc["NFL"] = comparisons[5], comparisons[5]
    p_values["NBA"].loc["NHL"], p_values["NHL"].loc["NBA"] = comparisons[0], comparisons[0]
    p_values["NBA"].loc["MLB"], p_values["MLB"].loc["NBA"] = comparisons[3], comparisons[3]
    p_values["NHL"].loc["MLB"], p_values["MLB"].loc["NHL"] = comparisons[1], comparisons[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [20]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
